# Keras基本例からNumerai


In [23]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
np.random.seed(1337)  # for reproducibility

import os

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

## 使用データ

同じフォルダにnumeraiのtrainingデータセットおくこと.

In [2]:
inputCSV = pd.read_csv('numerai_training_data.csv')
inputCSV.head(10)

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,target
0,0.801621,0.954027,0.570531,0.144233,0.050049,0.078679,0.929508,0.171078,0.883034,0.179370,...,0.199107,0.968784,0.064581,0.916191,0.238189,0.093656,0.206537,0.160289,0.659445,1
1,0.540071,0.459007,0.933132,0.499801,0.437993,0.267236,0.497195,0.456021,0.609753,0.187454,...,0.129508,0.479954,0.416888,0.634561,0.710521,0.324297,0.339858,0.349510,0.432921,1
2,0.518214,0.383788,0.424871,0.792763,0.774286,0.451385,0.791401,0.845301,0.118626,0.419257,...,0.266524,0.648017,0.420798,0.004319,0.992130,0.718081,0.992094,0.134067,0.074308,0
3,0.806454,0.161343,0.636976,0.201716,0.420538,0.151819,0.305147,0.117885,0.610892,0.249714,...,0.205248,0.086210,0.532110,0.790242,0.198782,0.274051,0.192057,0.097587,0.669891,0
4,0.415003,0.170186,0.418823,0.471830,0.570765,0.213789,0.158155,0.011730,0.418276,0.558754,...,0.506977,0.300006,0.742479,0.206817,0.814364,0.331907,0.065424,0.406144,0.746500,0
5,0.704449,0.850339,0.455998,0.224563,0.699544,0.834721,0.747288,0.901346,0.497416,0.903353,...,0.834001,0.729096,0.810783,0.609452,0.192695,0.621665,0.737860,0.194186,0.364996,1
6,0.788056,0.054596,0.200827,0.067378,0.941077,0.778948,0.116750,0.228003,0.136388,0.964285,...,0.784283,0.039394,0.933946,0.669043,0.138145,0.918439,0.880196,0.070342,0.479384,1
7,0.188298,0.604075,0.018341,0.815766,0.518478,0.803098,0.711386,0.652814,0.052646,0.817774,...,0.730338,0.586008,0.416888,0.223151,0.626276,0.963009,0.689730,0.689253,0.132086,0
8,0.820315,0.637409,0.907437,0.243122,0.378766,0.257982,0.983338,0.355089,0.883293,0.123529,...,0.221233,0.812422,0.112858,0.709823,0.598131,0.041301,0.500063,0.266616,0.735886,1
9,0.798261,0.968608,0.902818,0.304248,0.574142,0.225799,0.767128,0.287060,0.726069,0.001582,...,0.048475,0.904284,0.648878,0.630587,0.344180,0.457721,0.339951,0.158732,0.706181,1


In [3]:
# データの並びをシャッフル
calcData = inputCSV.reindex(np.random.permutation(inputCSV.index))
calcData.head(10)

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,target
48473,0.443313,0.202047,0.433925,0.900588,0.676421,0.994305,0.015536,0.922447,0.145433,0.815773,...,0.917480,0.028845,0.625409,0.391161,0.755612,0.842537,0.776601,0.962662,0.177580,0
34953,0.413209,0.587450,0.554613,0.229968,0.389487,0.225453,0.418001,0.263405,0.678585,0.466968,...,0.499500,0.443977,0.424711,0.526795,0.209773,0.316402,0.411231,0.398339,0.609873,1
60858,0.615468,0.055227,0.585383,0.960165,0.242121,0.101084,0.509524,0.290930,0.663911,0.228861,...,0.371125,0.200191,0.042157,0.741037,0.931261,0.179951,0.192057,0.841595,0.680841,1
37041,0.247645,0.252975,0.472430,0.895060,0.266913,0.305077,0.096784,0.180987,0.401190,0.613721,...,0.535187,0.152590,0.489294,0.008503,0.929211,0.424409,0.402792,0.652253,0.441397,0
14422,0.473702,0.621110,0.485596,0.923814,0.483345,0.755059,0.317458,0.912570,0.201371,0.649508,...,0.658438,0.229924,0.340058,0.443024,0.952947,0.645014,0.620591,0.984150,0.035088,0
32729,0.642131,0.464782,0.341651,0.851780,0.149229,0.506105,0.400209,0.627115,0.264778,0.378553,...,0.485076,0.459087,0.064581,0.940547,0.622032,0.590624,0.640839,0.646938,0.244663,0
85191,0.956559,0.892117,0.144696,0.442286,0.080573,0.026581,0.976715,0.055495,0.697097,0.518840,...,0.116615,0.952311,0.136662,0.876693,0.363951,0.100704,0.255280,0.687254,0.723152,1
78786,0.796721,0.827191,0.562501,0.400240,0.045727,0.025473,0.876220,0.102423,0.989937,0.060900,...,0.157466,0.904173,0.129123,0.789514,0.611513,0.024785,0.015996,0.306764,0.967408,1
24399,0.260473,0.160570,0.925190,0.711899,0.184976,0.191392,0.122635,0.031412,0.820566,0.207951,...,0.068264,0.033113,0.017698,0.092299,0.696832,0.151081,0.095900,0.839673,0.922540,1
77574,0.342186,0.283447,0.751640,0.159726,0.388537,0.270639,0.190381,0.508265,0.890597,0.155405,...,0.258841,0.241161,0.398368,0.840574,0.364830,0.045031,0.041518,0.273403,0.946607,0


In [4]:
# 入力データと教師データに分割
pd_y = calcData['target']
pd_x = calcData.drop(['target'], axis=1)

# numpy配列にデータフレームから変換
x_array = pd_x.as_matrix()
y_array = pd_y.as_matrix()

In [5]:
pd_x.shape[0]

96320

In [6]:
# 正規化し,float32の型に変換
x_scale = scale(x_array).astype(np.float32)

# サンプルの一部をテストデータ用に分割
N = pd_x.shape[0] * 0.1
X_train, X_test = np.split(x_scale, [N])

/Users/kohno/.pyenv/versions/3.5.1/envs/devkeras/lib/python3.5/site-packages/numpy/lib/shape_base.py:422: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  sub_arys.append(_nx.swapaxes(sary[st:end], axis, 0))


### 教師データのラベリング
変換が必要な点に注意.

In [7]:
batch_size = 2000
nb_classes = 2 # 2値分類
nb_epoch = 200

In [8]:
y_scale = y_array.astype(np.int32)
Y_train, Y_test = np.split(y_scale, [N])

# 離散の分類問題の場合、ベクトルのラベルをto_categoricalで変換しておく必要がある.
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(Y_train, nb_classes)
Y_test = np_utils.to_categorical(Y_test, nb_classes)

/Users/kohno/.pyenv/versions/3.5.1/envs/devkeras/lib/python3.5/site-packages/numpy/lib/shape_base.py:422: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  sub_arys.append(_nx.swapaxes(sary[st:end], axis, 0))


## Deep learningのモデル定義

以下では暫定的に多層ニューラルネットを用いる.

(ここが基本的に変更する場所である.)

なお、活性化関数にReLUを用いる.

また途中に過学習を防ぎ汎化性能を上げる手法であるdropoutが仕込んである.

In [54]:
model = Sequential()
model.add(Dense(80, input_shape=(21,)))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(80))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(2))
model.add(Activation('sigmoid'))

### モデルは一度コンパイルする形式を取るタイプ

サマリーが出てちょっと見やすい.

In [55]:
model.summary()

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_5 (Dense)                  (None, 80)            1760        dense_input_2[0][0]              
____________________________________________________________________________________________________
activation_5 (Activation)        (None, 80)            0           dense_5[0][0]                    
____________________________________________________________________________________________________
dropout_4 (Dropout)              (None, 80)            0           activation_5[0][0]               
____________________________________________________________________________________________________
dense_6 (Dense)                  (None, 100)           8100        dropout_4[0][0]                  
___________________________________________________________________________________________

In [56]:
es_cb = keras.callbacks.EarlyStopping(monitor='val_acc', patience=50, verbose=1, mode='auto')
log_filepath = os.curdir
#tb_cb = keras.callbacks.TensorBoard(log_dir=log_filepath, histogram_freq=1)

### 学習と結果

fitで学習し,evaluateでtestデータから汎化性能をチェックする.

In [57]:
history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_test, Y_test), callbacks=[es_cb])#, tb_cb])
#keras.callbacks.EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')
score = model.evaluate(X_test, Y_test, verbose=0)

print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 9632 samples, validate on 86688 samples
Epoch 1/200
9632/9632 [==============================] - 0s - loss: 0.7233 - acc: 0.5022 - val_loss: 0.6938 - val_acc: 0.5113
Epoch 2/200
9632/9632 [==============================] - 0s - loss: 0.7049 - acc: 0.5128 - val_loss: 0.6933 - val_acc: 0.5125
Epoch 3/200
9632/9632 [==============================] - 0s - loss: 0.7030 - acc: 0.5103 - val_loss: 0.6931 - val_acc: 0.5136
Epoch 4/200
9632/9632 [==============================] - 0s - loss: 0.7002 - acc: 0.5077 - val_loss: 0.6929 - val_acc: 0.5145
Epoch 5/200
9632/9632 [==============================] - 0s - loss: 0.6987 - acc: 0.5096 - val_loss: 0.6928 - val_acc: 0.5135
Epoch 6/200
9632/9632 [==============================] - 0s - loss: 0.6985 - acc: 0.4984 - val_loss: 0.6927 - val_acc: 0.5150
Epoch 7/200
9632/9632 [==============================] - 0s - loss: 0.6962 - acc: 0.5119 - val_loss: 0.6927 - val_acc: 0.5150
Epoch 8/200
9632/9632 [==============================] - 0s - loss: 0

In [58]:
pred = pd.read_csv("numerai_tournament_data.csv")
pred.head(10)

,t_id,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21
0,33032,0.919136,0.402220,0.335447,0.408911,0.909374,0.875515,0.793247,0.735592,0.290002,...,0.823239,0.859311,0.586265,0.509609,0.768567,0.024489,0.939576,0.596841,0.454167,0.188507
1,20838,0.072862,0.085398,0.691351,0.917969,0.184682,0.499284,0.025199,0.115818,0.781117,...,0.135286,0.253641,0.019908,0.128603,0.094148,0.858657,0.567579,0.605002,0.882755,0.960829
2,17585,0.268903,0.470812,0.178869,0.687795,0.622515,0.605008,0.832596,0.527291,0.202580,...,0.615814,0.611614,0.552004,0.631388,0.430585,0.447971,0.435045,0.320062,0.702936,0.266742
3,4231,0.291499,0.100183,0.108386,0.832005,0.202355,0.478545,0.166413,0.368036,0.209103,...,0.329354,0.374242,0.123975,0.482507,0.780585,0.432857,0.383226,0.509814,0.770422,0.341161
4,33357,0.151855,0.067270,0.236928,0.600259,0.892697,0.611378,0.081095,0.617280,0.336961,...,0.897775,0.390264,0.040880,0.788739,0.562621,0.816385,0.459390,0.446632,0.293370,0.215816
5,17425,0.243205,0.620738,0.884520,0.993783,0.148403,0.636627,0.358717,0.545112,0.583144,...,0.025126,0.245494,0.488298,0.269465,0.219151,0.960993,0.751257,0.663983,0.847568,0.606422
6,27310,0.433634,0.129248,0.457350,0.500149,0.988647,0.980214,0.062949,0.975227,0.076214,...,0.866592,0.828941,0.026036,0.957695,0.726683,0.570812,0.683132,0.793599,0.259133,0.073043
7,16522,0.478147,0.239203,0.472099,0.705242,0.636406,0.272477,0.076986,0.626842,0.465995,...,0.812237,0.499135,0.272245,0.925725,0.238305,0.875119,0.435045,0.583181,0.674225,0.378057
8,25340,0.690443,0.733230,0.669029,0.149143,0.166520,0.086494,0.542064,0.171097,0.594200,...,0.058273,0.133734,0.752680,0.103004,0.615365,0.040236,0.184330,0.163866,0.090175,0.509583
9,4840,0.651826,0.621780,0.636015,0.211715,0.816277,0.476272,0.507397,0.680482,0.558007,...,0.878738,0.595791,0.675861,0.753440,0.341442,0.322559,0.384265,0.687706,0.183704,0.338938


In [59]:
pred_x = pred.drop(["t_id"],axis=1)
pred_x.head(3)

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21
0,0.919136,0.402220,0.335447,0.408911,0.909374,0.875515,0.793247,0.735592,0.290002,0.984229,...,0.823239,0.859311,0.586265,0.509609,0.768567,0.024489,0.939576,0.596841,0.454167,0.188507
1,0.072862,0.085398,0.691351,0.917969,0.184682,0.499284,0.025199,0.115818,0.781117,0.360633,...,0.135286,0.253641,0.019908,0.128603,0.094148,0.858657,0.567579,0.605002,0.882755,0.960829
2,0.268903,0.470812,0.178869,0.687795,0.622515,0.605008,0.832596,0.527291,0.202580,0.359053,...,0.615814,0.611614,0.552004,0.631388,0.430585,0.447971,0.435045,0.320062,0.702936,0.266742


In [60]:
np_pred = pred_x.as_matrix()

ans = model.predict(np_pred)

In [61]:
pd_ans = pd.DataFrame(ans)
pd_ans[1].head(3)

0    0.518254
1    0.502082
2    0.519519
Name: 1, dtype: float64

In [62]:
pred_csv = pd.concat([pred["t_id"],pd_ans[1]], axis=1)
pred_csv.columns = ["t_id", "probability"]

In [63]:
pred_csv.head(10)

,t_id,probability
0,33032,0.518254
1,20838,0.502082
2,17585,0.519519
3,4231,0.507580
4,33357,0.509699
5,17425,0.513522
6,27310,0.525883
7,16522,0.506156
8,25340,0.506088
9,4840,0.526192


In [64]:
pred_csv.to_csv("pred_file.csv", index=False)